# SWD6: High Performance Python

## Introduction

**Hang on, don’t optimise too early!**

There are `trade-offs` when you pursue faster code. For example, it may become more complex, use more memory, or become less readable. This is in addition to the optimisation process taking time and effort. 

So, before jumping into optimising code, check that:

- The code is correct?
  - Have you tested it?
  - Does it have documentation?
- Is optimisation really needed?

**First Things First**

- Have you tried more suitable algorithms and data structures?
- Have you tried vectorisation?
- Have you tried compiling the code?

**If that’s still not enough, then:**

- Try parallelisation.
- Try offloading the work to accelerators.

<div style='border:1px solid #999;padding:1em'>
<h4>Warm-up Exercise</h4>
<p>Arvind has inherited the code below from their supervisor. They are finding it very slow to run over a large batch of files.</p>
<p><b>What are some things Arvind could do to improve this code <em>before</em> they seek to optimize it?</b></p>
<p><em>Discuss with a partner or small group (5 mins)</em></p>
</div>

In [ ]:
import os
import re

directory = 'sim_results'
matches = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            for result in re.findall('.* (.*)\[(.*)\]:.*', contents, re.DOTALL):
                if filename not in matches:
                    matches.append(filename)

with open('matches.txt', 'w') as f:
    for match in matches:
        f.write(f"{match}\n")

## Benchmarking

Benchmarking is the practice of comparing performance metrics:

- Execution Time
- Allocated Memory

### `timeit` Module

> "This module provides a simple way to time small bits of Python code.
> It has both a Command-Line Interface as well as a callable one."
> 
> \- [timeit documentation](https://docs.python.org/3/library/timeit.html)

Can also be used as a magic command in notebooks.

#### Measuring Function Execution Time

In [1]:
%timeit range(100)

56.6 ns ± 0.537 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


#### Measuring Cell Execution Time

In [2]:
%%timeit
for x in range(100):
    pass

783 ns ± 40 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Functions

### Built-in functions

[Built-in function List](https://docs.python.org/3/library/functions.html)

In [3]:
nums = [num for num in range(1_000_000)]

In [4]:
%%timeit
count = 0
for num in nums:
    count += 1

24.6 ms ± 967 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
len(nums)

37.2 ns ± 0.301 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Homemade Function

In [6]:
def hypotenuse(x, y):
    x = abs(x)
    y = abs(y)
    t = min(x, y)
    x = max(x, y)
    t = t / x
    return x * (1 + t * t) ** 0.5

In [7]:
%timeit hypotenuse(3.0, 4.0)

351 ns ± 3.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Can you simplify the math/logic?!

In [8]:
def hypotenuse(x, y):
    return (x*x + y*y) ** 0.5

In [9]:
%timeit hypotenuse(3.0, 4.0)

137 ns ± 4.45 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### External Packages

In [10]:
import math

In [11]:
%timeit math.hypot(3.0, 4.0)

59.1 ns ± 2.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Additional Tip: Avoid the dot operation!

In [12]:
from math import hypot

In [13]:
%timeit hypot(3.0, 4.0)

40.3 ns ± 1.26 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Data types/structures

### Built-in Types: List versus Dictionary

In [14]:
import numpy as np

needles = [2, 3, 10, 99, 101]
haystack_list = list(range(2,10,2)) #play with list length

In [15]:
%%timeit
needles_found = 0
for needle in needles:
    if needle in haystack_list:
        needles_found += 1

19.1 ms ± 420 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
haystack_dict = {value: key for key, value in enumerate(haystack_list)}

In [17]:
%%timeit
needles_found = 0
for needle in needles:
    if needle in haystack_dict:
        needles_found += 1

267 ns ± 3.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### External Types: List versus Numpy Arrays

In [18]:
multiplication_list = np.random.randint(low=0, high=100, size=(1_000_000))

In [19]:
listA = list(range(1,1_000_000,3))
arrayA = np.array(listA)

In [20]:
%%timeit
listB=[]
for item in listA:
    listB.append(item*2)

19.6 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Extra Tip: list comprehension!

In [21]:
%%timeit
listB = [item*2 for item in listA]

14.4 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
arrayB = arrayA*2

150 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Vectorization

Gaussian example

In [23]:
import math

SQRT_2PI = np.float32((2.0 * math.pi) ** 0.5)

mean = 0.0
sigma = 1.0


def my_function(x, mean, sigma):
    """Compute the value of a Gaussian probability density function at x with given mean and sigma."""
    return math.exp(-0.5 * ((x - mean) / sigma) ** 2.0) / (sigma * SQRT_2PI)

In [24]:
x = np.random.uniform(-3.0, 3.0, size=1_000_000)

In [25]:
%%timeit
result = np.zeros_like(x)
for i in range(len(x)):
    result[i] = my_function(x[i], mean, sigma)

1.52 s ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Numpy Vectorize

In [26]:
vectorized_function = np.vectorize(my_function)

In [27]:
%%timeit
result = vectorized_function(x, mean, sigma)

1.38 s ± 65.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Profiling: `line_profiler`

In [28]:
import time


def relax():
    pass


def bottleneck():
    time.sleep(0.001)


def some_function():
    nums = [num for num in range(1_000)]
    bigger_nums = [num**2 for num in nums]
    bottleneck()
    bigger_nums.extend(bigger_nums)
    relax()

In [29]:
%load_ext line_profiler

In [30]:
%lprun -f some_function some_function()

Timer unit: 1e-06 s

Total time: 0.001401 s
File: /tmp/ipykernel_178976/3630047074.py
Function: some_function at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def some_function():
    13         1         64.0     64.0      4.6      nums = [num for num in range(1_000)]
    14         1        236.0    236.0     16.8      bigger_nums = [num**2 for num in nums]
    15         1       1078.0   1078.0     76.9      bottleneck()
    16         1         22.0     22.0      1.6      bigger_nums.extend(bigger_nums)
    17         1          1.0      1.0      0.1      relax()

## Challenge!

In [31]:
def primes(n):
    if n==2:
        return [2]
    elif n<2:
        return []

    s=list(range(3,n+1,2))
    mroot = n ** 0.5
    half=(n+1)//2-1
    i=0
    m=3

    while m <= mroot:
        if s[i]:
            j=(m*m-3)//2
            s[j]=0
            while j<half:
                s[j]=0
                j+=m
        i=i+1
        m=2*i+3

    primes = [2]
    for x in s:
        if x:
            primes.append(x)
    return primes

print(primes(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [32]:
%timeit primes(100)

3.25 µs ± 47.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Possibilities:

- replace homemade functions
- use the best variable type
- simplify the math
- improve the logic
- can we vectorize?

In [33]:
from math import sqrt

In [34]:
def primes(n):

    if n>2:
        s=list(range(3,n+1,2))

        mroot = sqrt(n)  # add sqrt function
        half=(n+1)//2-1
        i=0
        m=3

        while m <= mroot:
            if s[i]:
                j=(m*m-3)//2
                s[j]=0
                for k in range(j, half, m): # replace while
                  s[k]=0
            i=i+1
            m=2*i+3
        return [2]+[x for x in s if x] # list comprehension

    elif n==2: # If-else ladder
        return [2]
    else:
        return []

print(primes(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [35]:
%timeit primes(100)

2.71 µs ± 45.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Using numpy vectorize - not good in this case?

In [36]:
def primes(n):

    def is_prime(n):
        return n>1 and all(n%i for i in range(2, int(n**0.5)+1))

    vec_prime = np.vectorize(is_prime)
    
    numbers = np.array(range(n))
    mask = vec_prime(numbers)
    return numbers[mask]

primes(100)


array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [37]:
%timeit primes(100)

72.3 µs ± 789 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
